<a href="https://colab.research.google.com/github/divakar1993/conda-docs/blob/main/stable_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install diffusers pillow torch transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.9 MB/s eta 0:00:00


In [30]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import BertTokenizer
from diffusers import StableDiffusionPipeline
import torch.nn as nn
import os

# Define a custom dataset class for your image and text data
class ImageTextDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        text_file = os.path.splitext(img_name)[0] + '.txt'
        with open(text_file, 'r') as file:
            text = file.read().strip()
        encoded_text = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

        return image, encoded_text['input_ids'][0], encoded_text['attention_mask'][0]

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Initialize the dataset with your data directory and transformations
dataset = ImageTextDataset(root_dir='/content/drive/MyDrive/dataset', transform=transform)

# Create a data loader
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define your model architecture and other configurations
model_id = "runwayml/stable-diffusion-v1-5"
model = StableDiffusionPipeline.from_pretrained(model_id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define your loss function
loss_fn = nn.MSELoss()

# Define your optimizer and learning rate
optimizer = torch.optim.AdamW(model.text_encoder.parameters(), lr=1e-4)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [35]:

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    total_loss = 0.0  # Track the total loss for this epoch
    for images, input_ids, attention_masks in data_loader:
        images = images.to(device)
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)

        # Forward pass and loss calculation
        text_prompt = "A daisy image."
        outputs = model(images=images, input_text=input_ids, prompt=text_prompt)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [52]:
# Save the trained model's state dictionary
torch.save(model, 'full_model.pth')

In [54]:
import torch

# Load the pretrained model
model = torch.load('full_model.pth')



In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# Load the pretrained model (assuming a text-to-image model like Stable Diffusion)
model = torch.load('full_model.pth')


# Prepare the text prompt "A daisy image
prompt = "a daisy"

# Perform inference
with torch.no_grad():  # Disable gradient calculations
    output = model(prompt=prompt)

# The output is likely an image or a set of images
# Here's an example of how to handle and display the output image
output_image = output.images[0]
output_image.show()


  0%|          | 0/50 [00:00<?, ?it/s]